# 多項式的表示與運算
f(x) = a<sub>n</sub>x<sup>n</sup> + a<sub>n-1</sub>x<sup>n-1</sup> + ... + a<sub>1</sub>x + a<sub>0</sub>
如果使用array來表示多項式，則array的index代表x的次方，array的value代表該次方的係數。例如：
```python
# 多項式 3x^4 + 0x^3 + 2x^2 + 5x + 7
poly = [7, 5, 2, 0, 3]

# 多項式 2x^980 + 5x^50 + 7
poly = [7] + [0]*49 + [5] + [0]*929 + [2] # 注意這樣的表示法會浪費大量(97.7%)空間
```
>linked list來表示多項式，可以節省空間，只儲存非零係數的項
多項式 5x^2 + 4x + 2
多項式        5x + 5

![多項式用 linked list 表達](https://img-blog.csdn.net/20170318104025267?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvdTAxMzMwOTg3MA==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

In [ ]:
# 節點結構
class Node:
    def __init__(self, coef, exp):
        self._coef = coef  # 係數
        self._exp = exp    # 次方
        self._next = None  # 指向下一個節點

# 多項式鏈結串列結構
class PolynomialLinkedList:
    def __init__(self):
        self._head = None  # 多項式的頭節點

    # 在多項式中插入一個新項
    def insert_term(self, coef, exp):
        new_node = Node(coef, exp)
        if self._head is None or self._head._exp < exp:
            new_node._next = self._head
            self._head = new_node
        else:
            current = self._head
            while current._next is not None and current._next._exp >= exp:
                current = current._next
            if current._exp == exp:
                current._coef += coef
            else:
                new_node._next = current._next
                current._next = new_node

    # 顯示多項式
    def __str__(self):
        terms = []
        current = self._head
        while current is not None:
            terms.append(f"{current._coef}x^{current._exp}")
            current = current._next
        return " + ".join(terms) if terms else "0"

In [ ]:
foo = PolynomialLinkedList()
foo.insert_term(3, 4)
foo.insert_term(5, 2)
foo.insert_term(2, 4)  # 合併同類項
print(foo)  # 輸出: 5x^4 + 5x

In [ ]:
def add_polynomials(poly1, poly2):
    result = PolynomialLinkedList()
    p1 = poly1._head
    p2 = poly2._head

    while p1 is not None and p2 is not None:
        if p1._exp > p2._exp:
            result.insert_term(p1._coef, p1._exp)
            p1 = p1._next
        elif p1._exp < p2._exp:
            result.insert_term(p2._coef, p2._exp)
            p2 = p2._next
        else:
            sum_coef = p1._coef + p2._coef
            if sum_coef != 0:
                result.insert_term(sum_coef, p1._exp)
            p1 = p1._next
            p2 = p2._next

    while p1 is not None:
        result.insert_term(p1._coef, p1._exp)
        p1 = p1._next

    while p2 is not None:
        result.insert_term(p2._coef, p2._exp)
        p2 = p2._next

    return result

In [ ]:
# 多項式 5x^2 + 4x + 2
# 多項式        5x + 5

poly1 = PolynomialLinkedList()
poly1.insert_term(5, 2)
poly1.insert_term(4, 1)
poly1.insert_term(2, 0)
poly2 = PolynomialLinkedList()
poly2.insert_term(5, 1)
poly2.insert_term(5, 0)
result = add_polynomials(poly1, poly2)
print(result)  # 輸出: 5x^2 + 9x + 7

# 稀疏矩陣的表示
稀疏矩陣(sparse matrix)是指大部分元素為零的矩陣。  
例如：
```
0 0 3 0 0
0 0 0 0 0
0 4 0 0 0
0 0 0 5 0
```
使用二維array表示稀疏矩陣會浪費大量空間，因此我們可以使用linked list來表示稀疏矩陣。  
每個非零元素使用一個節點(node)來表示，節點包含三個資訊：值(value)、行(row)、列(column)。  
節點之間使用指標(pointer)連接，形成一個鏈結串列(linked list)。  
例如，上述稀疏矩陣可以表示為以下的linked list：
(3, 0, 2) -> (4, 2, 1) -> (5, 3, 3) -> None

每個節點表示一個非零元素，格式為(值, 行, 列)。 
這樣的表示法可以節省空間，只儲存非零元素，並且可以方便地進行矩陣運算。

In [ ]:
class SpareMatrixNode:
    def __init__(self, value, position):
        self._value = value
        self._position = position  # (row, col)
        self._next = None

class SparseMatrixLinkedList:
    def __init__(self):
        self._head = None

    def insert(self, value, position):
        if value == 0:
            return
        new_node = SpareMatrixNode(value, position)
        if self._head is None:
            self._head = new_node
        else:
            current = self._head
            while current._next is not None:
                current = current._next
            current._next = new_node

    def __str__(self):
        elements = []
        current = self._head
        while current is not None:
            elements.append(f"Value: {current._value} at Position: {current._position}")
            current = current._next
        return "\n".join(elements) if elements else "Empty Matrix"
    
    def to_dense(self, rows, cols):
        dense_matrix = [[0 for _ in range(cols)] for _ in range(rows)]
        current = self._head
        while current is not None:
            r, c = current._position
            dense_matrix[r][c] = current._value
            current = current._next
        return dense_matrix
    
sparse_matrix = SparseMatrixLinkedList()
sparse_matrix.insert(3, (0, 2))
sparse_matrix.insert(4, (2, 1))
sparse_matrix.insert(5, (3, 3))
print(sparse_matrix)
dense = sparse_matrix.to_dense(10, 10)
for row in dense:
    print(row)